<a href="https://colab.research.google.com/github/Ayush-Kori/7-day-dp-workshop/blob/main/Backtesting_a_year_on_year_seasonal_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
df = pd.read_table('SPX_20071123.txt')
df['Date']=df['Date'].round().astype('int')

In [2]:
df['Date']=pd.to_datetime(df['Date'], format='%Y%m%d')
df.set_index('Date', inplace=True)

In [3]:
eomPrice=df.resample('M').last()[:-1] # End of month  prices.  Need to remove last date because it isn't  really end of January.
monthlyRet=eomPrice.pct_change(1, fill_method=None)
positions=np.zeros(monthlyRet.shape)

/tmp/ipython-input-2770599154.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  eomPrice=df.resample('M').last()[:-1] # End of month  prices.  Need to remove last date because it isn't  really end of January.


In [4]:
for m in range(13, monthlyRet.shape[0]):
  hasData=np.where(np.isfinite(monthlyRet.iloc[m-12,:]))[0]
  sortidx=np.argsort(monthlyRet.iloc[m-12, hasData])
  badData=np.where(np.logical_not(np.isfinite(monthlyRet.iloc[m-1, hasData[sortidx]])))
  topN=np.floor(len(sortidx)/10).astype('int')
  positions[m-1, hasData[sortidx.values[np.arange(-topN,0)]]]=1

In [5]:
capital=np.nansum(np.array(pd.DataFrame(abs(positions)).shift()), axis=1)
capital[capital==0]=1
ret=np.nansum(np.array(pd.DataFrame(positions).shift()), axis=1)
capital[capital==0]=1

In [7]:
ret=np.nansum(np.array(pd.DataFrame(abs(positions)).shift())*np.array(monthlyRet), axis=1)/capital
ret=np.delete(ret, np.arange(13))
avgret=np.nanmean(ret)*12
sharpe=np.sqrt(12)*np.nanmean(ret)/np.nanstd(ret)
print('Avg ann return=%f Sharpe ratio=%f' % (avgret, sharpe))

Avg ann return=0.170872 Sharpe ratio=0.818157
